In [ ]:
"""
Letroso Simulation & Analysis Toolkit
====================================

A single-file, notebook‑friendly implementation that covers:
  • Portuguese dictionary download & cleansing (≤10‑letter words)
  • Core feedback logic (green / yellow / grey)
  • Three playing strategies
      – Random (baseline)
      – Elimination (candidate‑pruning)
      – Entropy / information‑gain (heuristic optimal)
  • Monte‑Carlo harness with optional multiprocessing
  • Confidence‑interval helper
  • Exhaustive toy‑validation utility
  • Experiment runner producing a tidy pandas DataFrame
  • Convenience plot helpers (histogram & CDF)
  • Minimal __main__ demo

Each logical section is delimited by `# %%` so the file doubles as a
Jupyter‑compatible *percent‑script* (you can `jupyter nbconvert` it or open
directly in VS Code / JupyterLab).

Requires Python ≥ 3.11 and the packages listed in `requirements.txt`.
"""

# %% Imports & basic setup ----------------------------------------------------
from __future__ import annotations

import pathlib
import random
import urllib.request
from collections import Counter, defaultdict
from dataclasses import dataclass
from typing import List, Tuple

import pandas as pd
import matplotlib.pyplot as plt
import unidecode  # strip accents
from scipy import stats

# Reproducibility -------------------------------------------------------------
RNG = random.Random(42)

# Colour codes ----------------------------------------------------------------
GREEN, YELLOW, GREY = "G", "Y", "-"

# Aliases ---------------------------------------------------------------------
Word = str
Feedback = Tuple[str, ...]  # e.g. ("G", "Y", "-", ...)
Strategy = "BaseStrategy"   # forward ref for type checker

# %% Portuguese dictionary ----------------------------------------------------
DATA_DIR = pathlib.Path("data")
DICT_PATH = DATA_DIR / "dict_ptbr.txt"
DICT_URL = (
    "https://raw.githubusercontent.com/"
    "pythonprobr/palavras/master/palavras.txt"
)


def _download_dictionary(dest: pathlib.Path, url: str = DICT_URL) -> None:
    dest.parent.mkdir(exist_ok=True)
    print("⬇  Downloading Portuguese word list …")
    urllib.request.urlretrieve(url, dest)
    print("✓ Saved to", dest)


def load_dictionary_pt(max_len: int = 10, asciify: bool = True) -> List[Word]:
    """Return uppercase Portuguese words ≤ *max_len* letters."""
    if not DICT_PATH.exists():
        _download_dictionary(DICT_PATH)

    words: list[Word] = []
    with open(DICT_PATH, encoding="utf-8") as fh:
        for w in fh:
            w = w.strip()
            if 0 < len(w) <= max_len and w.isalpha():
                if asciify:
                    w = unidecode.unidecode(w)
                words.append(w.upper())
    return words


DICT: List[Word] = load_dictionary_pt()
print(f"Dictionary ready – {len(DICT):,} words ≤10 letters (accents removed)")

# %% Core game logic ----------------------------------------------------------

def feedback(secret: Word, guess: Word) -> Feedback:
    """Return tuple of colour codes for *guess* compared to *secret*."""
    if len(secret) != len(guess):
        raise ValueError("Secret and guess must have the same length")

    result = [GREY] * len(secret)
    remaining = Counter(secret)

    # Pass 1 – mark greens
    for i, (s, g) in enumerate(zip(secret, guess)):
        if s == g:
            result[i] = GREEN
            remaining[g] -= 1

    # Pass 2 – mark yellows
    for i, (s, g) in enumerate(zip(secret, guess)):
        if result[i] == GREY and remaining[g] > 0:
            result[i] = YELLOW
            remaining[g] -= 1

    return tuple(result)


# Self‑check ------------------------------------------------------------------
assert feedback("BANANA", "BACIAS") == (
    GREEN,
    GREEN,
    GREY,
    GREY,
    YELLOW,
    GREY,
)

# %% Strategy hierarchy -------------------------------------------------------
@dataclass
class BaseStrategy:
    """Uniform‑random baseline. Override *update* for smarter play."""

    dictionary: List[Word]

    # Runtime attributes (populated by *reset*)
    candidates: List[Word] = None  # type: ignore
    history: List[Tuple[Word, Feedback]] = None  # type: ignore

    def reset(self, word_len: int) -> None:
        self.candidates = [w for w in self.dictionary if len(w) == word_len]
        self.history = []

    def next_guess(self) -> Word:
        return RNG.choice(self.candidates)

    def update(self, guess: Word, fb: Feedback) -> None:
        self.history.append((guess, fb))


class EliminationStrategy(BaseStrategy):
    """Keeps only words consistent with all past feedback (Knuth‑style)."""

    def _consistent(self, word: Word, guess: Word, fb: Feedback) -> bool:
        return feedback(word, guess) == fb

    def update(self, guess: Word, fb: Feedback) -> None:
        super().update(guess, fb)
        self.candidates = [w for w in self.candidates if self._consistent(w, guess, fb)]


class EntropyStrategy(EliminationStrategy):
    """Selects guess that minimises expected remaining search space."""

    SAMPLE_LIMIT = 2000  # speed trade‑off for large candidate sets

    def next_guess(self) -> Word:
        best_word: Word | None = None
        best_score = float("inf")

        pool = (
            self.candidates
            if len(self.candidates) <= self.SAMPLE_LIMIT
            else RNG.sample(self.candidates, self.SAMPLE_LIMIT)
        )

        for word in pool:
            bucket_sizes: defaultdict[Feedback, int] = defaultdict(int)
            for secret in self.candidates:
                bucket_sizes[feedback(secret, word)] += 1
            exp_remaining = sum(n * n for n in bucket_sizes.values()) / len(self.candidates)
            if exp_remaining < best_score:
                best_word, best_score = word, exp_remaining

        return best_word or RNG.choice(self.candidates)


# %% Simulation engine --------------------------------------------------------
@dataclass
class GameResult:
    secret: Word
    attempts: int
    won: bool


def play_game(secret: Word, strategy: Strategy, max_attempts: int = 10) -> GameResult:
    strategy.reset(len(secret))
    for attempt in range(1, max_attempts + 1):
        guess = strategy.next_guess()
        fb = feedback(secret, guess)
        if all(c == GREEN for c in fb):
            return GameResult(secret, attempt, True)
        strategy.update(guess, fb)
    return GameResult(secret, max_attempts, False)


# %% Monte‑Carlo harness & CI -------------------------------------------------

def run_many(
    n_games: int,
    strat_cls,
    dictionary: List[Word] = DICT,
    n_jobs: int = 1,
) -> pd.DataFrame:
    """Simulate *n_games* and return a DataFrame (one row per game)."""

    if n_jobs == 1:
        results = [
            play_game(RNG.choice(dictionary), strat_cls(dictionary)) for _ in range(n_games)
        ]
    else:
        import multiprocessing as mp

        with mp.Pool(n_jobs) as pool:
            results = pool.starmap(
                play_game,
                [
                    (RNG.choice(dictionary), strat_cls(dictionary))
                    for _ in range(n_games)
                ],
            )

    return pd.DataFrame([r.__dict__ for r in results])


def ci_mean(series: pd.Series, alpha: float = 0.05) -> tuple[float, float]:
    m = series.mean()
    h = stats.t.ppf(1 - alpha / 2, len(series) - 1) * series.std(ddof=1) / (len(series) ** 0.5)
    return m - h, m + h


# %% Validation utility -------------------------------------------------------

def exhaustive_validation() -> None:
    """Run analytic vs simulated win‑rate on an exhaustive toy dictionary."""

    toy = ["AAA", "AAB", "ABA", "ABB", "BAA", "BAB", "BBA", "BBB"]
    analytic = 1 - (1 - 1 / len(toy)) ** 10  # random guesses w/ replacement
    df = run_many(50_000, BaseStrategy, toy)
    sim = df["won"].mean()
    print(
        f"Validation toy‑dict ⇒ analytic={analytic:.4f}  simulation={sim:.4f}  Δ={(sim-analytic):+.4f}"
    )


# %% Experiment & plots -------------------------------------------------------

def experiment(replications: int = 20_000, n_jobs: int = 1) -> pd.DataFrame:
    scenarios = {
        "Random": BaseStrategy,
        "Elimination": EliminationStrategy,
        "Entropy": EntropyStrategy,
    }
    rows: list[dict] = []
    for name, strat in scenarios.items():
        df = run_many(replications, strat, n_jobs=n_jobs)
        mu = df["attempts"].mean()
        ci_low, ci_high = ci_mean(df["attempts"])
        rows.append(
            dict(
                Strategy=name,
                MeanAttempts=mu,
                CI_low=ci_low,
                CI_high=ci_high,
                WinRate=df["won"].mean(),
            )
        )
    return pd.DataFrame(rows)


def plot_hist(df: pd.DataFrame, label: str) -> None:
    df["attempts"].hist(bins=range(1, 12), rwidth=0.8)
    plt.title(f"Distribution of attempts – {label}")
    plt.xlabel("Attempts")
    plt.ylabel("Frequency")
    plt.show()


def plot_cdf(df_a: pd.DataFrame, df_b: pd.DataFrame, label_a: str, label_b: str) -> None:
    plt.figure()
    for df, label in ((df_a, label_a), (df_b, label_b)):
        data = sorted(df["attempts"].to_list())
        y = [i / len(data) for i in range(1, len(data) + 1)]
        plt.step(data, y, where="post", label=label)
    plt.xlabel("Attempts")
    plt.ylabel("CDF")
    plt.legend()
    plt.grid(True)
    plt.title("CDF of attempts")
    plt.show()


# %% Main demo ----------------------------------------------------------------
if __name__ == "__main__":
    print("Running sanity demo …")
    df_demo = run_many(5_000, EliminationStrategy)
    print(df_demo.head())
    print("Mean attempts :", df_demo["attempts"].mean())
    print("Win rate      :", df_demo["won"].mean())

    print("\nQuick experiment (Random × Elimination × Entropy)")
    print(experiment(replications=5_000))

    # Uncomment for validation & plots
    # exhaustive_validation()
    # plot_hist(run_many(10_000, BaseStrategy), "Random")
    # plot_hist(run_many(10_000, EliminationStrategy), "Elimination"

⬇  Downloading Portuguese word list …
✓ Saved to data\dict_ptbr.txt
Dictionary ready – 177,953 words ≤10 letters (accents removed)
Running sanity demo …
       secret  attempts   won
0    TREMELAL         3  True
1    ESCANSAO         5  True
2    ALOQUIRO         3  True
3  ENGALANEAR         3  True
4      TURURI         4  True
Mean attempts : 3.6786
Win rate      : 0.9988

Quick experiment (Random × Elimination × Entropy)
